<a href="https://colab.research.google.com/github/Melanie279/Algoritmos_Paralelos/blob/master/Paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!nvidia-smi


Wed Oct 21 14:36:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
#include "cuda_runtime.h"  // Definicion de tipo de integrados y superposiciones de funciones para las extensiones del lenguaje CUDA
#include <stdio.h> // cabezera input-output
#include <stdlib.h> //biblioteca estandar
#include <cuda.h> //define el host publico funciones y tipos para la API de controlador CUDA

#define MAXVALUE 10000

//------------------------------------

void numberGen(int N, int max, int* store) //Funcion Generar numeros
{
    int i;
    srand(time(0)); //tiempo para generar numeros aleatorios
    for (i = 0; i < N; i++)
        store[i] = rand() % max; //Almacena en el arreglo
}
//------------------------------------

__global__ void countOdds(int* d, int N, int* odds)
{
    #asignamos memoria compartida dinamicamente(extern significa quela matriz hace referencia a la memoria compartida declarada en otro lugar)
    extern __shared__ int count[];

    int myID = blockIdx.x * blockDim.x + threadIdx.x; #ñadiendo el tamaño del grid y del bloque 
    int localID = threadIdx.x; #ponemos el id en cada hilo
    count[localID] = 0;
    if (myID < N)
        count[localID] = (d[myID] % 2);
    __syncthreads();

    // fase de reducción 
    int step = 1;
    while (((localID | step) < blockDim.x) && ((localID & step) == 0))
    {
        count[localID] += count[localID | step];
        step <<= 1;
        __syncthreads();
    }

   
    // adiciona al contador global
    if (localID == 0)
        atomicAdd(odds, count[0]);
}

//------------------------------------
int sharedSize(int b)//Generar Tamaño compartido
{
    return b * sizeof(int);//Devuelve la cantidad de memoria asignada
}
//------------------------------------

int main(int argc, char** argv) //Funcion principal
{
    
    int N = 5000; //Definimos la cantidad de numeros que se generaran aleatoriamente
    int* ha, * hres, * da, * dres;   // punteros host (h*) y device (d*)

    ha = new int[N];
    hres = new int[1];

    cudaMalloc((void**)&da, sizeof(int) * N);# Almacena un espacio en la memoria para da
    cudaMalloc((void**)&dres, sizeof(int) * 1);#Almacena un espacio en la memoria para dres

    numberGen(N, MAXVALUE, ha);

    cudaMemcpy(da, ha, sizeof(int) * N, cudaMemcpyHostToDevice); # copia datos a la memoria del dispositivo
    cudaMemset(dres, 0, sizeof(int)); #llenamos los primeros bytes de recuento del área de memoria apuntada por dres con el valor de byte valor constante.

    int blockSize, gridSize;
 #Devuelve el tamaño de cuadrícula y bloque que logra la máxima ocupación potencial para una función de dispositivo.
    cudaOccupancyMaxPotentialBlockSizeVariableSMem(&gridSize, &blockSize, (void*)countOdds, sharedSize, N);

    gridSize = ceil(1.0 * N / blockSize);
    printf("Grid : %i    Block : %i\n", gridSize, blockSize);
    countOdds << < gridSize, blockSize, blockSize * sizeof(int) >> > (da, N, dres);

    cudaMemcpy(hres, dres, sizeof(int), cudaMemcpyDeviceToHost);# copia datos a la memoria del dispositivo
    // chequeo
    int oc = 0;
    for (int i = 0; i < N; i++)
        if (ha[i] % 2){
            printf("%i ",ha[i]);
            oc++;
        }
    printf("\n");
            
    printf("Cantidad de numeros impares - Chequeo\n");
    printf("%i %i\n", *hres, oc);

    cudaFree((void*)da); #Libera memoria de da
    cudaFree((void*)dres);#Libera memoria de dres
    delete[]ha;
    delete[]hres;
    cudaDeviceReset();
    getchar();
    return 0;
}

Grid : 5    Block : 1024
33 2857 3253 5907 2261 6077 8915 8699 8547 2739 6175 6907 1749 799 151 389 2965 6389 4775 7413 2677 827 4543 4541 7445 959 8141 3069 3699 8303 3869 5787 259 901 5207 4063 127 6345 7807 1655 8767 8623 4725 2939 1053 8181 2051 5217 9567 463 1253 8559 9121 777 969 3153 9787 9259 6757 3793 2031 5115 4493 5507 5619 8995 425 3585 1783 7633 2163 8033 7689 9887 9147 9797 8969 247 9807 6711 8803 6381 9935 5279 9797 3951 8975 559 8747 4395 8039 6647 5659 9527 3247 8225 4339 9767 533 1051 201 5409 6351 3535 9221 9649 3783 8381 6091 2777 3657 797 8767 6187 2033 1617 5287 5991 9671 4801 7415 4569 851 9119 2447 6669 7235 1343 8955 8345 379 6659 8865 7149 4077 9591 2695 9261 8413 9747 5295 1091 7279 7021 3961 7427 5745 1243 2831 5897 957 1101 8171 9161 1047 9359 4457 4365 5511 4679 6487 2975 9667 3421 5859 1313 6593 4245 5841 2665 135 9565 7915 3431 731 5961 6751 8379 1977 3755 7573 5243 945 2589 2573 1075 303 4669 5211 2985 9401 7189 1947 5305 183 1565 91 9399 8753 6099 9633